<a href="https://colab.research.google.com/github/gnmergen/occupancy_detection/blob/main/Occupancy_Detection_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Packages Used:

In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn

In [ ]:
# turned out we didn't need gpu
cuda0 = torch.device('cuda:0')  # pick the GPU at index 0

In [ ]:
# Preprocessing into NN friendly format

occupancy = pd.read_csv('occupancy_data.csv')
occupancy = occupancy.drop('date', axis = 1)
occupancy['Occupancy'] = occupancy['Occupancy'].astype(object)

In [ ]:
X, Y = occupancy.drop('Occupancy', axis = 1), occupancy['Occupancy']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20) # splitting data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # normalizing predictor variables
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
y_train = y_train.to_numpy()

In [ ]:
y_test = y_test.to_numpy()

In [ ]:
class Classifier(nn.Module):
  def __init__(self, input_dimension):
    super(Classifier, self).__init__()
    self.fc1 = nn.Linear(input_dimension, 32)
    self.fc2 = nn.Linear(32, 32)
    self.fc3 = nn.Linear(32, 16)
    self.fc4 = nn.Linear(16, 1)

  def forward(self, input_dimension):
    input_dimension = self.fc1(input_dimension)
    input_dimension = input_dimension.relu()
    input_dimension = self.fc2(input_dimension)
    input_dimension = input_dimension.relu()
    input_dimension = self.fc3(input_dimension)
    input_dimension = input_dimension.relu()
    return self.fc4(input_dimension)

_, input_dimension = X_train.shape
model = Classifier(input_dimension)

In [ ]:
def configure_loss_function():
  return nn.BCEWithLogitsLoss()

def configure_optimizer(model):
  return torch.optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
X_train = torch.from_numpy(X_train.astype(np.float32)) # Numpy to Tensor for PyTorch
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32)).reshape(-1,1)
y_test = torch.from_numpy(y_test.astype(np.float32)).reshape(-1,1)

In [ ]:
criterion = configure_loss_function()
optimizer = configure_optimizer(model)

In [ ]:
def full_gd(model, criterion, optimizer, X_train, y_train, epochs = 5000):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)
  
  for it in range(epochs):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    outputs_test = model(X_test)
    loss_test = criterion(outputs_test, y_test)
    
    train_losses[it] = loss.item()
    test_losses[it] = loss_test.item()
    
    if (it + 1) % 20 == 0:
      print(f"Epoch {it+1}/{epochs}, Training Loss: {loss.item():.4f}, Test Loss: {loss_test.item():.4f}")
  return train_losses, test_losses

In [ ]:
train_losses, test_losses = full_gd(model, criterion, optimizer, X_train, y_train)

Epoch 20/5000, Training Loss: 0.0437, Test Loss: 0.0536
Epoch 40/5000, Training Loss: 0.0436, Test Loss: 0.0534
Epoch 60/5000, Training Loss: 0.0434, Test Loss: 0.0533
Epoch 80/5000, Training Loss: 0.0433, Test Loss: 0.0532
Epoch 100/5000, Training Loss: 0.0432, Test Loss: 0.0531
Epoch 120/5000, Training Loss: 0.0431, Test Loss: 0.0530
Epoch 140/5000, Training Loss: 0.0430, Test Loss: 0.0529
Epoch 160/5000, Training Loss: 0.0430, Test Loss: 0.0527
Epoch 180/5000, Training Loss: 0.0429, Test Loss: 0.0526
Epoch 200/5000, Training Loss: 0.0428, Test Loss: 0.0525
Epoch 220/5000, Training Loss: 0.0427, Test Loss: 0.0524
Epoch 240/5000, Training Loss: 0.0426, Test Loss: 0.0523
Epoch 260/5000, Training Loss: 0.0425, Test Loss: 0.0522
Epoch 280/5000, Training Loss: 0.0424, Test Loss: 0.0521
Epoch 300/5000, Training Loss: 0.0424, Test Loss: 0.0520
Epoch 320/5000, Training Loss: 0.0423, Test Loss: 0.0519
Epoch 340/5000, Training Loss: 0.0422, Test Loss: 0.0518
Epoch 360/5000, Training Loss: 0.04

In [ ]:
with torch.no_grad():
  p_train = model(X_train)
  p_train = (p_train.numpy() > 0)

  train_acc = np.mean(y_train.numpy() == p_train)

  p_test = model(X_test)
  p_test = (p_test.numpy() > 0)

  test_acc = np.mean(y_test.numpy() == p_test)

print(f'Training Accuracy is: {train_acc*100:.2f}%')
print(f'Testing Accuracy is: {test_acc*100:.2f}%')

Training Accuracy is: 98.92%
Testing Accuracy is: 98.61%
